<a href="https://colab.research.google.com/github/tasanyphy01773/Machine_Learning/blob/main/3_Training_and_Testing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install -q sklearn

In [32]:
%tensorflow_version 2.x

In [33]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from IPython.display import clear_output 
from six.moves import urllib 
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf 

In [34]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') #train data 
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') #test data 
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [35]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [36]:
CATAGORICAL_COLUMNS = ['sex', 'n_siblings_spouses','parch', 'class', 
                       'deck', 'embark_town', 'alone']

NUMERICAL_COLUMNS = ['age', 'fare']

feature_columns = []

for feature_name in CATAGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERICAL_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype= tf.float32))

In [37]:
feature_columns

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.strin

In [38]:
#creating input function
"""
All we did here is, input training and testing data, selected epoch and shuffle 
"""

def make_input_fn(data_df, label_df, num_epochs = 15, shuffle = True, batch_size = 32):
  def input_function(): #inner function this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)  #split dataset and repeat by epoch 
    return ds #returned a batch of dataset
  return input_function


train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs =1, shuffle = False)

In [39]:
#creating model
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output() #clear console output 
print(result)

print(result['accuracy'])

{'accuracy': 0.74242425, 'accuracy_baseline': 0.625, 'auc': 0.832262, 'auc_precision_recall': 0.7941703, 'average_loss': 0.49471635, 'label/mean': 0.375, 'loss': 0.48735204, 'precision': 0.63247865, 'prediction/mean': 0.4530039, 'recall': 0.74747473, 'global_step': 300}
0.74242425


In [40]:
result = list(linear_est.predict(eval_input_fn))
clear_output()
# print(result)
print(dfeval.loc[2])
print(y_eval.loc[2])
print(result[2]['probabilities'][1]) # 1: survived

sex                        female
age                            58
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
1
0.8504544


In [41]:
result = list(linear_est.predict(eval_input_fn))
n= 100
clear_output()
# print(result)
print(dfeval.loc[n])
print(y_eval.loc[n])
print(result[n]['probabilities'][1]) # 1: survived

sex                          male
age                            30
n_siblings_spouses              0
parch                           0
fare                         7.25
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 100, dtype: object
0
0.12019067
